## Libraries

In [1]:
import os
import rioxarray as rxr
from shapely.geometry import mapping
import geopandas as gpd
import rasterio as rio

C:\Users\enhi\AppData\Local\Temp\ipykernel_4784\3461951.py:4: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


In [7]:
main_dir = r'C:\Users\enhi\OneDrive - gmv.com\Documents\Projects\ANIN\Generating Indices\SMA'

monthly_SMA = os.path.join(main_dir, 'SMA_monthly')
# Output directory
Clipped_path = os.path.join(main_dir, 'Clipped_SMA')
# Directory for boundary shp
Boundary = os.path.join(main_dir, 'Boundary')
# CRS of the project
Copernicus_ex =rxr.open_rasterio(r'C:\Users\enhi\OneDrive - gmv.com\Documents\Projects\ANIN\Generating Indices\FAPAR anomaly\Monthly_FAPAR_Glass\April\2000April.tif', masked =True)
Copernicus_ex_crs = Copernicus_ex.rio.crs

In [8]:
# Load the shapefile
def load_shape_file(filepath):
    """Loads the shape file desired to mask a grid.
    Args:
        filepath: Path to *.shp file
    """
    shpfile = gpd.read_file(filepath)
    print("""Shapefile loaded. To prepare for masking, run the function
        `select_shape`.""")
    return shpfile

#Create the mask
def select_shape(shpfile):

    """Select the submask of interest from the shapefile.
    Args:
        shpfile: (*.shp) loaded through `load_shape_file`
        category: (str) header of shape file from which to filter shape.
            (Run print(shpfile) to see options)
        name: (str) name of shape relative to category.
           Returns:
        shapely polygon
    """

    col_code = 'ISO3_CODE'
    country_codes = ['ZAF', 'LSO', 'SWZ']

    # Extract the rows that have 'ZAF', 'LSO', or 'SWZ' in the 'SOV_A3' column
    selected_rows = shpfile[shpfile[col_code].isin(country_codes)]

    # Combine the selected polygons into a single polygon
    unioned_polygon = selected_rows.geometry.unary_union

    # Convert the unioned polygon to a geopandas dataframe with a single row
    mask_polygon = gpd.GeoDataFrame(geometry=[unioned_polygon])
    
    print("""Mask created.""")

    return mask_polygon
shpfile = load_shape_file(Boundary + '\\'+ 'CNTR_RG_01M_2020_4326.shp')
AOI = select_shape(shpfile)
# Define a crs to the AOI 
AOI.crs =Copernicus_ex_crs

Shapefile loaded. To prepare for masking, run the function
        `select_shape`.
Mask created.


In [14]:
# looping through SMA images
for image in os.listdir(monthly_SMA):
    # path to the image
    path = os.path.join(monthly_SMA, image)
    # open the image as xarray
    SMA =rxr.open_rasterio(path, masked = True)
    # Clipping to the AOI
    clipped = SMA.rio.clip(AOI.geometry.apply(mapping),
                                     crs=AOI.crs,
                                     all_touched=True,
                                     from_disk=True).squeeze()
    # Export clipped images with the name of month and year
    clipped.rio.to_raster(Clipped_path + '\\'+ image[12:18]+ '.tif')